# Deep learning 101 with Pytorch and fastai

> Some code and text snippets have been extracted from the book [\"Deep Learning for Coders with Fastai and Pytorch: AI Applications Without a PhD\"](https://course.fast.ai/), and from these blog posts [[ref1](https://muellerzr.github.io/fastblog/2021/02/14/Pytorchtofastai.html)].

In [ ]:
from fastcore.all import *

In [ ]:
# export
import torch
from torch.utils.data import TensorDataset
import matplotlib.pyplot as plt
import torch.nn as nn

In [ ]:
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
device

## Linear regression model in Pytorch

### Datasets and Dataloaders

We'll create a dataset that contains $(x,y)$ pairs sampled from the linear function $y = ax + b+ \epsilon$. To do this, we'll create a PyTorch's `TensorDataset`.

A PyTorch tensor is nearly the same thing as a NumPy array. The vast majority of methods and operators supported by NumPy on these structures are also supported by PyTorch, but PyTorch tensors have additional capabilities. One major capability is that these structures can live on the GPU, in which case their computation will be optimized for the GPU and can run much faster (given lots of values to work on). In addition, PyTorch can automatically calculate derivatives of these operations, including combinations of operations. These two things are critical for deep learning

In [ ]:
@delegates(plt.Axes.scatter)
def show_TensorFunction1D(x, y, y_hat=None, return_fig=False, **kwargs):
    r"""
        Displays the 1D function `y`(`x`), with optional predictions `y_hat`
    """
    figure = plt.figure()
    ax = figure.add_subplot(111)
    ax.scatter(x, y, **kwargs)
    if y_hat is not None: ax.scatter(x, y_hat, **kwargs)
    if return_fig: return figure


def linear_function_dataset(a, b, n=100, show_plot=False):
    r"""
        Creates a Pytorch's `TensorDataset` with `n` random samples of the 
        linear function y = `a`*x + `b`. `show_plot` dcides whether or not to 
        plot the dataset
    """
    x = torch.randn(n, 1)
    y = a*x + b + 0.1*torch.randn(n, 1)
    if show_plot:
        show_TensorFunction1D(x, y, marker='.')
    return TensorDataset(x, y)

In [ ]:
a = 2
b = 3
n = 100
data = linear_function_dataset(a, b, n, show_plot=True)
test_eq(type(data), TensorDataset)

In every machine/deep learning experiment, we need to have at least two datasets:
 - training: used to train the model
 - validation: used to validate the model after each training step. It allows to detect overfitting and adjust the hyperparameters of the model properly

In [ ]:
train_ds = linear_function_dataset(a, b, n=100, show_plot=True)
valid_ds = linear_function_dataset(a, b, n=20, show_plot=True)

A dataloader combines a dataset an a sampler that samples data into **batches**, and provides an iterable over the given dataset. 

In [ ]:
bs = 10
train_dl = torch.utils.data.DataLoader(train_ds, batch_size=bs, shuffle=True)
valid_dl = torch.utils.data.DataLoader(valid_ds, batch_size=bs, shuffle=False)

In [ ]:
for i, data in enumerate(train_dl, 1):
    x, y = data
    print(f'batch {i}: x={x.shape} ({x.device}), y={y.shape} ({y.device})')

### Defining a linear regression model in Pytorch

The class `torch.nn.Module` is the base structure for all models in Pytorch. It mostly helps to register all the trainable parameters. A module is an object of a class that inherits from the PyTorch `nn.Module` class.

To implement an `nn.Module` you just need to:

- Make sure the superclass __init__ is called first when you initialize it.
- Define any parameters of the model as attributes with nn.Parameter. To tell `Module` that we want to treat a tensor as a parameter, we have to wrap it in the `nn.Parameter` class. All PyTorch modules use `nn.Parameter` for any trainable parameters. This class doesn't actually add any functionality (other than automatically calling `requires_grad`). It's only used as a "marker" to show what to include in parameters:
- Define a forward function that returns the output of your model.

In [ ]:
#export
class LinRegModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.a = nn.Parameter(torch.randn(1))
        self.b = nn.Parameter(torch.randn(1))
        
    def forward(self, x): return self.a*x + self.b

In [ ]:
model = LinRegModel()
pa, pb = model.parameters()
pa, pa.shape, pb, pb.shape

Objects of this class behave identically to standard Python functions, in that you can call them using parentheses and they will return the activations of a model.

In [ ]:
x = torch.randn(10, 1)
out = model(x)
x, x.shape, out, out.shape 

### Loss function and optimizer

The loss is the thing the machine is using as the measure of performance to decide how to update model parameters. The loss function is simple enough for a regression problem, we'll just use the Mean Square Error (MSE)

In [ ]:
loss_func = nn.MSELoss()
loss_func(x, out)

We have data, a model, and a loss function; we only need one more thing we can fit a model, and that's an optimizer.

In [ ]:
opt_func = torch.optim.SGD(model.parameters(), lr = 1e-3)

### Training loop

During training, we need to push our model and our batches to the GPU. Calling `cuda()` on a model or a tensor this class puts all these parameters on the GPU:

In [ ]:
model = model.to(device)

To train a model, we will need to compute all the gradients of a given loss with respect to its parameters, which is known as the *backward pass*. The *forward pass* is where we compute the output of the model on a given input, based on the matrix products. PyTorch computes all the gradients we need with a magic call to `loss.backward`. The backward pass is the chain rule applied multiple times, computing the gradients from the output of our model and going back, one layer at a time. 

In Pytorch, Each basic function we need to differentiate is written as a `torch.autograd.Function` object that has a `forward` and a `backward` method. PyTorch will then keep trace of any computation we do to be able to properly run the backward pass, unless we set the `requires_grad` attribute of our tensors to `False`. 

For minibatch gradient descent (the usual way of training in deep learning), we calculate gradients on batches. Before moving onto the next batch, we modify our model's parameters based on the gradients. For each iteration through our dataset (which would be called an **epoch**), the optimizer would perform as many updates as we have batches.

There are two important methods in a Pytorch optimizer:
- `zero_grad`: In PyTorch, we need to set the gradients to zero before starting to do backpropragation because PyTorch accumulates the gradients on subsequent backward passes. `zero_grad` just loops through the parameters of the model and sets the gradients to zero. It also calls `detach_`, which removes any history of gradient computation, since it won't be needed after `zero_grad`.

In [ ]:
n_epochs = 10

In [ ]:
# export
def train(model, device, train_dl, loss_func, opt_func, epoch_idx):
    r"""
        Train `model` for one epoch, whose index is given in `epoch_idx`. The 
        training loop will iterate through all the batches of `train_dl`, using
        the the loss function given in `loss_func` and the optimizer given in `opt_func`
    """
    running_loss = 0.0
    batches_processed = 0
    for batch_idx, (x, y) in enumerate(train_dl, 1):
        x, y = x.to(device), y.to(device) # Push data to GPU

        opt_func.zero_grad() # Reset gradients
        # Forward pass
        output = model(x)
        loss = loss_func(output, y)
        # Backward pass
        loss.backward()
        # Optimizer step
        opt_func.step()

        # print statistics
        running_loss += loss.item()
        batches_processed += 1
    print(f'Train loss [Epoch {epoch_idx}]: {running_loss/batches_processed : .2f})')

In [ ]:
for epoch in range(1, n_epochs+1):
    train(model, device, train_dl, loss_func, opt_func, epoch)

We can see how the parameters of the regression model are getting closer to the truth values `a` and `b` from the linear function.

In [ ]:
L(model.named_parameters())

### Validating the model

Validating the model requires only a forward pass, it's just inference. Disabling gradient calculation with the method `torch.no_grad()` is useful for inference, when you are sure that you will not call :meth:`Tensor.backward()`. 

In [ ]:
def validate(model, device, dl):
    running_loss = 0.
    total_batches = 0
    with torch.no_grad():
        for x, y in valid_dl:
            x, y = x.to(device), y.to(device)
            output = model(x)
            loss = loss_func(output, y)
            running_loss += loss.item()
            total_batches += 1

    print(f'Valid loss: {running_loss/total_batches : .2f}')

In [ ]:
validate(model, device, valid_dl)

In order to spot overfitting, it is useful to validate the model after each training epoch. 

In [ ]:
for epoch in range(1, n_epochs +1):
    train(model, device, train_dl, loss_func, opt_func, epoch)
    validate(model, device, valid_dl)

## Abstracting the manual training loop: moving from Pytorch to fastai

In [ ]:
from fastai.basics import *
from fastai.callback.progress import ProgressCallback

We can entirely replace the custom training loop with fastai's. That means you can get rid of `train()`, `validate()`, and the epoch loop in the original code, and replace it all with a couple of lines.

fastai's training loop lives in a `Learner`. The Learner is the glue that merges everything together (Datasets, Dataloaders, model and optimizer) and enables to train by just calling a `fit` function.

fastai's `Learner` expects DataLoaders to be used, rather than simply one DataLoader, so let's make that. We could just do `dls = Dataloaders(train_dl, valid_dl)`, to keep the PyTorch Dataloaders. However, by using a fastai `DataLoader` instead, created directly from the `TensorDataset` objects, we have some automations, such as automatic pushing of the data to GPU. 

In [ ]:
dls = DataLoaders.from_dsets(train_ds, valid_ds, bs=10)

In [ ]:
learn = Learner(dls, model=LinRegModel(), loss_func=nn.MSELoss(), opt_func=SGD)

Now we have everything needed to do a basic `fit`:

In [ ]:
default_device(0)

In [ ]:
learn.fit(10, lr=1e-3)

Having a Learner allows us to easily gather the model predictions for the validation set, which we can use for visualisation and analysis

In [ ]:
inputs, preds, outputs = learn.get_preds(with_input=True)
inputs.shape, preds.shape, outputs.shape

In [ ]:
show_TensorFunction1D(inputs, outputs, y_hat=preds, marker='.')

## Building a simple neural network

For the next example, we will create the dataset by sampling values from a non linear sample $y(x) = -\frac{1}{100}x^7 - x^4 - 2x^2 - 4x + 1$

In [ ]:
# export
def nonlinear_function_dataset(n=100, show_plot=False):
    r"""
        Creates a Pytorch's `TensorDataset` with `n` random samples of the 
        nonlinear function y = (-1/100)*x**7 -x**4 -2*x**2 -4*x + 1 with a bit 
        of noise. `show_plot` decides whether or not to plot the dataset
    """
    x = torch.rand(n, 1)*20 - 10 # Random values between [-10 and 10]
    y = (-1/100)*x**7 -x**4 -2*x**2 -4*x + 1 + 0.1*torch.randn(n, 1)
    if show_plot:
        show_TensorFunction1D(x, y, marker='.')
    return TensorDataset(x, y)

In [ ]:
n = 100
ds = nonlinear_function_dataset(n, show_plot=True)
x, y = ds.tensors
test_eq(x.shape, y.shape)

We will create the trainin and test dataset, and build the Dataloaders with them, this time directly in fastai, using the `Dataloaders.from_dsets` method.

In [ ]:
train_ds = nonlinear_function_dataset(n=1000)
valid_ds = nonlinear_function_dataset(n=200)

Normalization in deep learning are used to make optimization easier by smoothing the loss surface of the network. We will normalize the data based on the mean and std of the train dataset

In [ ]:
norm_mean = train_ds.tensors[1].mean()
norm_std = train_ds.tensors[1].std()

In [ ]:
train_ds_norm = TensorDataset(train_ds.tensors[0],
                              (train_ds.tensors[1] - norm_mean)/norm_std)
valid_ds_norm = TensorDataset(valid_ds.tensors[0],
                              (valid_ds.tensors[1] - norm_mean)/norm_std)

In [ ]:
dls = DataLoaders.from_dsets(train_ds_norm, valid_ds_norm, bs = 32)

We will build a Multi Layer Perceptron with 3 hidden layers. These networks are also known as Feed-Forward Neural Networks. The layers aof this type of networks are known as Fully Connected Layers, because, between every subsequent pair of layers, all the neurons are connected to each other.

<img alt="Neural network architecture" caption="Neural network" src="https://i.imgur.com/5ZWPtRS.png">

The easiest way of wrapping several layers in Pytorch is using the `nn.Sequential` module. It creates a module with a `forward` method that will call each of the listed layers or functions in turn, without us having to do the loop manually in the forward pass.

In [ ]:
class MLP3(nn.Module):
    r"""
        Multilayer perceptron with 3 hidden layers, with sizes `nh1`, `nh2` and
        `nh3` respectively.
    """
    def __init__(self, n_in=1, nh1=200, nh2=100, nh3=50, n_out=1):
        super().__init__()
        self.layers = nn.Sequential(
            nn.Linear(n_in, nh1),
            nn.ReLU(), 
            nn.Linear(nh1, nh2),
            nn.ReLU(),
            nn.Linear(nh2, nh3),
            nn.ReLU(),
            nn.Linear(nh3, n_out)
        )
        
    def forward(self, x): return self.layers(x)

In [ ]:
x, y = dls.one_batch()
model = MLP3()
output = model(x)
output.shape

In [ ]:
learn = Learner(dls, MLP3(), loss_func=nn.MSELoss(), opt_func=Adam)

In [ ]:
learn.fit(10, lr=1e-3)

In [ ]:
inputs, preds, outputs = learn.get_preds(with_input = True)
show_TensorFunction1D(inputs, outputs, y_hat=preds, marker='.')

Let's compare these results with the ones by our previous linear regression model

In [ ]:
learn_lin = Learner(dls, LinRegModel(), loss_func=nn.MSELoss(), opt_func=Adam)

In [ ]:
learn_lin.fit(20, lr=1e-3)

In [ ]:
inputs, preds, outputs = learn_lin.get_preds(with_input = True)
show_TensorFunction1D(inputs, outputs, y_hat=preds, marker='.')